In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# GPU training sucks on Mac, as usual
tf.config.set_visible_devices([], 'GPU')

**Exporting SavedModels**

In [3]:
from pathlib import Path

# extra code – load and split the MNIST dataset
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# extra code – build & train an MNIST model (also handles image preprocessing)
tf.random.set_seed(42)
tf.keras.backend.clear_session()
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

Epoch 1/10
1719/1719 [==============================] - 1s 601us/step - loss: 0.6725 - accuracy: 0.8312 - val_loss: 0.3669 - val_accuracy: 0.9002
Epoch 2/10
1719/1719 [==============================] - 1s 518us/step - loss: 0.3500 - accuracy: 0.9029 - val_loss: 0.2985 - val_accuracy: 0.9160
Epoch 3/10
1719/1719 [==============================] - 1s 515us/step - loss: 0.3013 - accuracy: 0.9146 - val_loss: 0.2647 - val_accuracy: 0.9250
Epoch 4/10
1719/1719 [==============================] - 1s 517us/step - loss: 0.2712 - accuracy: 0.9234 - val_loss: 0.2423 - val_accuracy: 0.9302
Epoch 5/10
1719/1719 [==============================] - 1s 522us/step - loss: 0.2484 - accuracy: 0.9297 - val_loss: 0.2231 - val_accuracy: 0.9360
Epoch 6/10
1719/1719 [==============================] - 1s 518us/step - loss: 0.2297 - accuracy: 0.9355 - val_loss: 0.2095 - val_accuracy: 0.9394
Epoch 7/10
1719/1719 [==============================] - 1s 522us/step - loss: 0.2140 - accuracy: 0.9405 - val_loss: 0.1954 -

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 rescaling (Rescaling)       (None, 784)               0         
                                                                 
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


**Installing and starting TensorFlow Serving**

In [ ]:
%% bash --bg
docker run -it --rm -v ".//my_mnist_model:/models/my_mnist_model" \
    -p 8500:8500 -p 8501:8501 -e MODEL_NAME=my_mnist_model tensorflow/serving


**Quering TF Serving through the REST API**

In [6]:
import json

X_new = X_test[:3]  # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [7]:
request_json

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 84, 185, 159, 151, 60, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 222, 254, 254, 254, 254, 241, 198, 198, 198, 198, 198, 198, 198, 198, 170, 52, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 67, 114, 72, 114, 163, 227, 254, 225, 254, 254, 254, 250, 229, 254, 254, 140, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 

In [8]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()  # raise an exception in case of error
response = response.json()

In [9]:
response

{'predictions': [[0.000160335447,
   2.99201844e-07,
   0.000446828693,
   0.00469595753,
   1.54922247e-06,
   5.59761e-05,
   5.45345493e-08,
   0.994365096,
   1.2006617e-05,
   0.000261799723],
  [0.00117343478,
   5.60543085e-05,
   0.981231511,
   0.00797711592,
   1.99952535e-08,
   0.00111281173,
   0.00659136847,
   7.62433761e-09,
   0.00185747142,
   1.58857944e-07],
  [2.89319814e-05,
   0.974329,
   0.00953976344,
   0.00242033112,
   0.000519438181,
   0.00151151652,
   0.0020808829,
   0.00650305394,
   0.00284645194,
   0.000220586749]]}

In [10]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

In [11]:
y_test[:3]

array([7, 2, 1], dtype=uint8)

**Quering TF Serving through the gRPC API**

In [14]:
tf.make_tensor_proto(X_new)

dtype: DT_UINT8
tensor_shape {
  dim {
    size: 3
  }
  dim {
    size: 28
  }
  dim {
    size: 28
  }
}
tensor_content: "\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000T\271\237\227<$\000\000\000\000\000\000\000\000\000\000\000\000\000\

In [16]:
# Create a PredictRequest protocol buffer and fill in the required fields

from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]  # == "flatten_input"
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [17]:
# Send the request to the server and get its response
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

In [19]:
# Convert the PredictResponse protocol buffer to a tensor
output_name = model.output_names[0]  # == "dense_1"
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)
y_proba.round(2)


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

**Deploying a new model version**

In [20]:
# extra code – build and train a new MNIST model version
np.random.seed(42)
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8),
    tf.keras.layers.Rescaling(scale=1 / 255),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 1s 530us/step - loss: 0.7172 - accuracy: 0.8060 - val_loss: 0.3362 - val_accuracy: 0.9090
Epoch 2/10
1719/1719 [==============================] - 1s 493us/step - loss: 0.3179 - accuracy: 0.9090 - val_loss: 0.2696 - val_accuracy: 0.9244
Epoch 3/10
1719/1719 [==============================] - 1s 497us/step - loss: 0.2683 - accuracy: 0.9230 - val_loss: 0.2316 - val_accuracy: 0.9348
Epoch 4/10
1719/1719 [==============================] - 1s 533us/step - loss: 0.2365 - accuracy: 0.9320 - val_loss: 0.2122 - val_accuracy: 0.9394
Epoch 5/10
1719/1719 [==============================] - 1s 543us/step - loss: 0.2127 - accuracy: 0.9385 - val_loss: 0.1921 - val_accuracy: 0.9440
Epoch 6/10
1719/1719 [==============================] - 1s 496us/step - loss: 0.1934 - accuracy: 0.9439 - val_loss: 0.1785 - val_accuracy: 0.9514
Epoch 7/10
1719/1719 [==============================] - 1s 535us/step - loss: 0.1773 - accuracy: 0.9489 - val_loss: 0.1644 -

In [21]:
model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


**Creating a Prediction Service on Vertex AI**

In [72]:
import os
import sys

project_id = "handson-ml3"  ##### CHANGE THIS TO YOUR PROJECT ID #####

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()
elif "kaggle_secrets" in sys.modules:
    from kaggle_secrets import UserSecretsClient
    UserSecretsClient().set_gcloud_credentials(project=project_id)
else:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/nikolaoschachampis/gitrepos/handson-ml3/ml-new.json"

In [73]:
from google.cloud import storage

bucket_name = "ml-nch"  ##### CHANGE THIS TO A UNIQUE BUCKET NAME #####
location = "eu-southwest1"

storage_client = storage.Client(project=project_id)
# bucket = storage_client.create_bucket(bucket_name, location=location)
bucket = storage_client.bucket(bucket_name)  # to reuse a bucket instead

In [82]:
list(bucket.list_blobs())

[<Blob: ml-nch, my_cifar10_model/fingerprint.pb, 1698329119852512>,
 <Blob: ml-nch, my_cifar10_model/saved_model.pb, 1698329120292037>,
 <Blob: ml-nch, my_cifar10_model/variables/variables.data-00000-of-00001, 1698329109935343>,
 <Blob: ml-nch, my_cifar10_model/variables/variables.index, 1698329099275400>,
 <Blob: ml-nch, my_mnist_model/0001/fingerprint.pb, 1718980929425595>,
 <Blob: ml-nch, my_mnist_model/0001/keras_metadata.pb, 1718980929635627>,
 <Blob: ml-nch, my_mnist_model/0001/saved_model.pb, 1718980929926677>,
 <Blob: ml-nch, my_mnist_model/0001/variables/variables.data-00000-of-00001, 1718980930511225>,
 <Blob: ml-nch, my_mnist_model/0001/variables/variables.index, 1718980930711349>,
 <Blob: ml-nch, my_mnist_model/0002/fingerprint.pb, 1718980928114866>,
 <Blob: ml-nch, my_mnist_model/0002/keras_metadata.pb, 1718980928360650>,
 <Blob: ml-nch, my_mnist_model/0002/saved_model.pb, 1718980928635519>,
 <Blob: ml-nch, my_mnist_model/0002/variables/variables.data-00000-of-00001, 17189

In [33]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

In [81]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'/Users/nikolaoschachampis/gitrepos/handson-ml3/ml-new.json'

In [83]:
# # I can'get past this stupid permissions error
# from google.cloud import aiplatform

# server_image = "gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-8:latest"

# aiplatform.init(project=project_id, location="europe-west8")
# mnist_model = aiplatform.Model.upload(
#     display_name="mnist",
#     artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
#     serving_container_image_uri=server_image,
# )

AttributeError: 'NoneType' object has no attribute 'from_call'

### Deploying a Model to a Mobile or Embedded device

In [80]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # 8-bit quantization
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

2024-06-23 19:16:41.501429: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-06-23 19:16:41.501441: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-06-23 19:16:41.501576: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_mnist_model/0002
2024-06-23 19:16:41.501988: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-06-23 19:16:41.501997: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: my_mnist_model/0002
2024-06-23 19:16:41.503131: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-06-23 19:16:41.519843: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: my_mnist_model/0002
2024-06-23 19:16:41.525439: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Too

### Using GPUs to Speed Up Computations

In [84]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [86]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

**Placing Operations and Variables on Devices**

In [87]:
a = tf.Variable([1., 2., 3.])
a.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [88]:
# This won't work if we have hiddedn the gpu
with tf.device("/gpu:0"):
    c = tf.Variable([1., 2., 3.])

c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [ ]:
tf.config.threading.set_intra_op_parallelism_threads()